In [ ]:
!pip install -r requirements.txt

In [ ]:
from accelerate.utils import write_basic_config

write_basic_config()

In [ ]:
!accelerate launch train.py --mixed_precision "fp16" --gradient_accumulation_steps 1 --learning_rate "1e-02" --gradient_checkpointing --max_grad_norm 1 --checkpointing_steps 100000 --max_train_steps=15000 --use_ema --lr_scheduler "constant" --lr_warmup_steps 0 --center_crop --random_flip --pretrained_model_name_or_path "runwayml/stable-diffusion-v1-5" --dataset_name "nroggendorff/zelda" --resolution 512 --train_batch_size 1 --output_dir "sd-zelda-model"

In [ ]:
from huggingface_hub import upload_folder, notebook_login, create_repo

In [ ]:
notebook_login()

In [ ]:
repo_id = create_repo(
    repo_id="nroggendorff/zelda-diffusion", exist_ok=True
).repo_id

In [ ]:
upload_folder(
    repo_id=repo_id,
    folder_path="sd-zelda-model",
    commit_message=f"Steps 10k",
    ignore_patterns=["step_*", "epoch_*"],
)

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipeline = StableDiffusionPipeline.from_pretrained("sd-zelda-model", torch_dtype=torch.float16, use_safetensors=True).to("cuda")

image = pipeline(prompt="a woman holding a sword").images[0]
image.save("zelda.png")